In [6]:
# necessary imports
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import pingouin as pg
from statsmodels.stats.anova import AnovaRM
from sklearn.preprocessing import OneHotEncoder
import statsmodels.api as sm
import statsmodels.formula.api as smf
from io import StringIO

# Load Data

In [2]:
P = {} # df per Participant
data_dir = "clean_data"
for csv_file in os.listdir(data_dir):
    if csv_file.startswith("P"): # Only get Participant Directories
        pn = int(csv_file[1:-4])
        tmp = pd.read_csv(os.path.join(data_dir,csv_file))
        tmp["participant"] = pn
        P[pn] = tmp.dropna()
df = pd.concat(P.values(), axis = 0)# all data

# Fixed Effects Models

In [7]:
models = {}
for emo_col in ["stress","valence","arousal","attention"]: # Loop over each dv
    dv = f"binary_{emo_col}"
    df[dv] = df[emo_col].apply(lambda x: 0 if x <=0 else 1) # Turn dependent variables into binary data
    formula = f"{dv} ~ C(app_category) * at_home + C(participant)" # Define fixed effects model
    models[emo_col] = smf.logit(formula=formula, data=df).fit() # Run model

Optimization terminated successfully.
         Current function value: 0.502883
         Iterations 7
         Current function value: 0.537154
         Iterations: 35


/Users/mauro/.pyenv/versions/3.10.6/envs/sandbox/lib/python3.10/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Optimization terminated successfully.
         Current function value: 0.526096
         Iterations 9
Optimization terminated successfully.
         Current function value: 0.582606
         Iterations 7


In [8]:
results = {}
for var, res in models.items():
    html_string = res.summary().tables[1].as_html() # Extract the HTML from the model summary
    html_io = StringIO(html_string) # Use StringIO to wrap the HTML string
    results[var] = pd.read_html(html_io, header=0)[0] # Read the HTML into a DataFrame

## Stress model

### Full table

In [16]:
stress_df = results["stress"][~results["stress"]['Unnamed: 0'].str.contains('participant')]
stress_df

,Unnamed: 0,coef,std err,z,P>|z|,[0.025,0.975]
0,Intercept,-1.9882,0.234,-8.513,0.000,-2.446,-1.530
1,C(app_category)[T.COMICS],0.0403,0.033,1.216,0.224,-0.025,0.105
2,C(app_category)[T.ENTERTAINMENT],0.1044,0.036,2.906,0.004,0.034,0.175
3,C(app_category)[T.GAME],0.0016,0.042,0.038,0.970,-0.081,0.085
4,C(app_category)[T.LIBRARIES_AND_DEMO],-0.0187,0.097,-0.193,0.847,-0.209,0.171
5,C(app_category)[T.MUSIC_AND_AUDIO],0.3397,0.035,9.695,0.000,0.271,0.408
6,C(app_category)[T.NEWS_AND_MAGAZINES],0.8564,0.113,7.593,0.000,0.635,1.077
7,C(app_category)[T.SHOPPING],0.0449,0.041,1.107,0.268,-0.035,0.124
8,C(app_category)[T.SOCIAL],0.1627,0.029,5.582,0.000,0.106,0.220
9,C(app_category)[T.SPORTS],0.1632,0.077,2.120,0.034,0.012,0.314


### Filtering for signigicant effects only p < .05

In [17]:
stress_df[stress_df["P>|z|"] < .05]

,Unnamed: 0,coef,std err,z,P>|z|,[0.025,0.975]
0,Intercept,-1.9882,0.234,-8.513,0.000,-2.446,-1.530
2,C(app_category)[T.ENTERTAINMENT],0.1044,0.036,2.906,0.004,0.034,0.175
5,C(app_category)[T.MUSIC_AND_AUDIO],0.3397,0.035,9.695,0.000,0.271,0.408
6,C(app_category)[T.NEWS_AND_MAGAZINES],0.8564,0.113,7.593,0.000,0.635,1.077
8,C(app_category)[T.SOCIAL],0.1627,0.029,5.582,0.000,0.106,0.220
9,C(app_category)[T.SPORTS],0.1632,0.077,2.120,0.034,0.012,0.314
10,C(app_category)[T.VIDEO_PLAYERS],0.2430,0.036,6.752,0.000,0.172,0.314
11,at_home[T.True],0.2123,0.089,2.385,0.017,0.038,0.387
88,C(app_category)[T.COMICS]:at_home[T.True],-0.3968,0.174,-2.284,0.022,-0.737,-0.056
89,C(app_category)[T.ENTERTAINMENT]:at_home[T.True],-0.3456,0.125,-2.775,0.006,-0.590,-0.102


## Valence model

### Full table

In [18]:
valence_df = results["valence"][~results["valence"]['Unnamed: 0'].str.contains('participant')]
valence_df

,Unnamed: 0,coef,std err,z,P>|z|,[0.025,0.975]
0,Intercept,1.7455,0.243,7.184,0.000,1.269,2.222
1,C(app_category)[T.COMICS],0.1810,0.034,5.247,0.000,0.113,0.249
2,C(app_category)[T.ENTERTAINMENT],-0.1296,0.037,-3.467,0.001,-0.203,-0.056
3,C(app_category)[T.GAME],0.1575,0.044,3.587,0.000,0.071,0.244
4,C(app_category)[T.LIBRARIES_AND_DEMO],0.0427,0.099,0.433,0.665,-0.150,0.236
5,C(app_category)[T.MUSIC_AND_AUDIO],0.0490,0.037,1.317,0.188,-0.024,0.122
6,C(app_category)[T.NEWS_AND_MAGAZINES],-0.3306,0.125,-2.650,0.008,-0.575,-0.086
7,C(app_category)[T.SHOPPING],0.0883,0.042,2.095,0.036,0.006,0.171
8,C(app_category)[T.SOCIAL],-0.0109,0.031,-0.356,0.722,-0.071,0.049
9,C(app_category)[T.SPORTS],-0.3394,0.077,-4.388,0.000,-0.491,-0.188


### Filtering for signigicant effects only p < .05

In [19]:
valence_df[valence_df["P>|z|"] < .05]

,Unnamed: 0,coef,std err,z,P>|z|,[0.025,0.975]
0,Intercept,1.7455,0.243,7.184,0.000,1.269,2.222
1,C(app_category)[T.COMICS],0.1810,0.034,5.247,0.000,0.113,0.249
2,C(app_category)[T.ENTERTAINMENT],-0.1296,0.037,-3.467,0.001,-0.203,-0.056
3,C(app_category)[T.GAME],0.1575,0.044,3.587,0.000,0.071,0.244
6,C(app_category)[T.NEWS_AND_MAGAZINES],-0.3306,0.125,-2.650,0.008,-0.575,-0.086
7,C(app_category)[T.SHOPPING],0.0883,0.042,2.095,0.036,0.006,0.171
9,C(app_category)[T.SPORTS],-0.3394,0.077,-4.388,0.000,-0.491,-0.188
10,C(app_category)[T.VIDEO_PLAYERS],0.1260,0.037,3.385,0.001,0.053,0.199
89,C(app_category)[T.ENTERTAINMENT]:at_home[T.True],0.7579,0.136,5.593,0.000,0.492,1.024
90,C(app_category)[T.GAME]:at_home[T.True],-0.3411,0.144,-2.374,0.018,-0.623,-0.060


## Arousal model

### Full table

In [20]:
arousal_df = results["arousal"][~results["arousal"]['Unnamed: 0'].str.contains('participant')]
arousal_df

,Unnamed: 0,coef,std err,z,P>|z|,[0.025,0.975]
0,Intercept,0.4250,0.174,2.439,0.015,0.083,0.766
1,C(app_category)[T.COMICS],0.2828,0.035,8.179,0.000,0.215,0.351
2,C(app_category)[T.ENTERTAINMENT],-0.1046,0.039,-2.703,0.007,-0.180,-0.029
3,C(app_category)[T.GAME],0.2311,0.042,5.519,0.000,0.149,0.313
4,C(app_category)[T.LIBRARIES_AND_DEMO],0.2709,0.095,2.845,0.004,0.084,0.457
5,C(app_category)[T.MUSIC_AND_AUDIO],-0.0648,0.037,-1.756,0.079,-0.137,0.008
6,C(app_category)[T.NEWS_AND_MAGAZINES],-0.2488,0.110,-2.259,0.024,-0.465,-0.033
7,C(app_category)[T.SHOPPING],-0.3056,0.043,-7.146,0.000,-0.389,-0.222
8,C(app_category)[T.SOCIAL],0.0773,0.031,2.479,0.013,0.016,0.138
9,C(app_category)[T.SPORTS],-0.2047,0.077,-2.646,0.008,-0.356,-0.053


### Filtering for signigicant effects only p < .05

In [21]:
arousal_df[arousal_df["P>|z|"] < .05]

,Unnamed: 0,coef,std err,z,P>|z|,[0.025,0.975]
0,Intercept,0.4250,0.174,2.439,0.015,0.083,0.766
1,C(app_category)[T.COMICS],0.2828,0.035,8.179,0.000,0.215,0.351
2,C(app_category)[T.ENTERTAINMENT],-0.1046,0.039,-2.703,0.007,-0.180,-0.029
3,C(app_category)[T.GAME],0.2311,0.042,5.519,0.000,0.149,0.313
4,C(app_category)[T.LIBRARIES_AND_DEMO],0.2709,0.095,2.845,0.004,0.084,0.457
6,C(app_category)[T.NEWS_AND_MAGAZINES],-0.2488,0.110,-2.259,0.024,-0.465,-0.033
7,C(app_category)[T.SHOPPING],-0.3056,0.043,-7.146,0.000,-0.389,-0.222
8,C(app_category)[T.SOCIAL],0.0773,0.031,2.479,0.013,0.016,0.138
9,C(app_category)[T.SPORTS],-0.2047,0.077,-2.646,0.008,-0.356,-0.053
10,C(app_category)[T.VIDEO_PLAYERS],0.1807,0.037,4.860,0.000,0.108,0.254


## Attention model

### Full table

In [22]:
attention_df = results["attention"][~results["attention"]['Unnamed: 0'].str.contains('participant')]
attention_df

,Unnamed: 0,coef,std err,z,P>|z|,[0.025,0.975]
0,Intercept,1.4145,0.219,6.474,0.000,0.986,1.843
1,C(app_category)[T.COMICS],0.2532,0.032,7.828,0.000,0.190,0.317
2,C(app_category)[T.ENTERTAINMENT],0.0396,0.034,1.149,0.251,-0.028,0.107
3,C(app_category)[T.GAME],-0.0428,0.040,-1.074,0.283,-0.121,0.035
4,C(app_category)[T.LIBRARIES_AND_DEMO],0.2300,0.095,2.417,0.016,0.043,0.416
5,C(app_category)[T.MUSIC_AND_AUDIO],0.0656,0.034,1.901,0.057,-0.002,0.133
6,C(app_category)[T.NEWS_AND_MAGAZINES],-0.0881,0.111,-0.797,0.426,-0.305,0.129
7,C(app_category)[T.SHOPPING],0.0454,0.039,1.150,0.250,-0.032,0.123
8,C(app_category)[T.SOCIAL],0.1470,0.028,5.173,0.000,0.091,0.203
9,C(app_category)[T.SPORTS],-0.1046,0.069,-1.516,0.129,-0.240,0.031


### Filtering for signigicant effects only p < .05

In [23]:
attention_df[attention_df["P>|z|"] < .05]

,Unnamed: 0,coef,std err,z,P>|z|,[0.025,0.975]
0,Intercept,1.4145,0.219,6.474,0.000,0.986,1.843
1,C(app_category)[T.COMICS],0.2532,0.032,7.828,0.000,0.190,0.317
4,C(app_category)[T.LIBRARIES_AND_DEMO],0.2300,0.095,2.417,0.016,0.043,0.416
8,C(app_category)[T.SOCIAL],0.1470,0.028,5.173,0.000,0.091,0.203
10,C(app_category)[T.VIDEO_PLAYERS],0.1691,0.035,4.854,0.000,0.101,0.237
11,at_home[T.True],0.2390,0.087,2.744,0.006,0.068,0.410
90,C(app_category)[T.GAME]:at_home[T.True],-0.7576,0.139,-5.469,0.000,-1.029,-0.486
91,C(app_category)[T.LIBRARIES_AND_DEMO]:at_home[...,-1.9009,0.802,-2.370,0.018,-3.473,-0.329
92,C(app_category)[T.MUSIC_AND_AUDIO]:at_home[T.T...,0.3225,0.139,2.314,0.021,0.049,0.596
95,C(app_category)[T.SOCIAL]:at_home[T.True],-0.2097,0.096,-2.191,0.028,-0.397,-0.022
